In [1]:
import os
print(os.getcwd())
os.chdir('/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1')
print(os.getcwd())

/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/src/benchmarking
/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1


In [2]:
import pandas as pd

corpus_df=pd.read_csv("data/benchmark_datasets/multihop_rag/corpus/corpus_chunks.csv")
corpus_df

,id,category,author,published_at,title,source,chunk
0,1,entertainment,NaN,2023-11-27 08:45:59+00:00,200+ of the best deals from Amazon's Cyber Mon...,Mashable,"table of contents table of contents echo , fir..."
1,2,entertainment,NaN,2023-11-27 08:45:59+00:00,200+ of the best deals from Amazon's Cyber Mon...,Mashable,amazon ' s invite - only deals for prime membe...
2,3,entertainment,NaN,2023-11-27 08:45:59+00:00,200+ of the best deals from Amazon's Cyber Mon...,Mashable,"for $ 229 . 99 ) , our new favorite ipad deal ..."
3,4,entertainment,NaN,2023-11-27 08:45:59+00:00,200+ of the best deals from Amazon's Cyber Mon...,Mashable,"2 ' s 11th gen , harness ##ing better speeds a..."
4,5,business,Stan Choe,2023-09-26 19:11:30+00:00,ASX set to drop as Wall Street’s September slu...,The Sydney Morning Herald,"et ##f provider beta ##sha ##res , which manag..."
...,...,...,...,...,...,...,...
4482,4483,technology,Natasha Lomas,2023-10-30 13:52:00+00:00,Meta to offer ad-free subscription in Europe i...,TechCrunch,meta is to offer an ad - free subscription ver...
4483,4484,technology,Natasha Lomas,2023-10-30 13:52:00+00:00,Meta to offer ad-free subscription in Europe i...,TechCrunch,"user ’ s accounts center . after march 1 , 202..."
4484,4485,technology,Natasha Lomas,2023-10-30 13:52:00+00:00,Meta to offer ad-free subscription in Europe i...,TechCrunch,protection authorities . in a press release ea...
4485,4486,technology,Natasha Lomas,2023-10-30 13:52:00+00:00,Meta to offer ad-free subscription in Europe i...,TechCrunch,implemented to the platforms as soon as pr ##a...


In [11]:
import pandas as pd
import os
from tqdm import tqdm
from src.extractors.relationship_extractor import extract_entities_and_relationships_from_df

def process_and_save_entities(df, text_column, output_entities_file, output_relationships_file, start_row=0):
    """
    逐行提取 DataFrame 中的实体和关系，并保存到文件中。

    参数:
    df (DataFrame): 包含文本数据的 DataFrame。
    output_entities_file (str): 保存实体结果的 CSV 文件路径。
    output_relationships_file (str): 保存关系结果的 CSV 文件路径。
    start_row (int): 从指定行开始处理（用于恢复中断的处理过程）。
    """
    # 如果文件已存在，加载已处理的部分
    if os.path.exists(output_entities_file):
        df_entities_existing = pd.read_csv(output_entities_file)
        df_relationships_existing = pd.read_csv(output_relationships_file)
    else:
        df_entities_existing = pd.DataFrame()
        df_relationships_existing = pd.DataFrame()
    
    # 使用 tqdm 显示进度条，设置从 start_row 开始
    for index, row in tqdm(df.iterrows(), total=len(df),  desc="Processing rows"):
        if index < start_row:
            continue  # 跳过已处理的部分

        try:
            # 提取实体和关系
            df_entities, df_relationships = extract_entities_and_relationships_from_df(pd.DataFrame([row]), text_column)
            
            # 将结果追加保存到文件中
            df_entities.to_csv(output_entities_file, mode='a', header=not os.path.exists(output_entities_file), index=False)
            df_relationships.to_csv(output_relationships_file, mode='a', header=not os.path.exists(output_relationships_file), index=False)
        
        except Exception as e:
            # 发生错误时打印出错误信息以及中断的行号
            print(f"中断于第 {index} 行，错误: {e}")
            break  # 停止处理
    
    print(f"完成处理 {index} 行.")

# 使用示例
output_entities_file = 'export/multihop_rag/entities.csv'
output_relationships_file = 'export/multihop_rag/relationships_output.csv'

# 记录程序中断的位置（如需要）
start_row = 627  # 如果程序中断后重启，设置为上次中断的行号 + 1

process_and_save_entities(corpus_df, "chunk", output_entities_file, output_relationships_file, start_row=start_row)


Processing rows:  14%|█▍        | 630/4487 [01:15<07:44,  8.30it/s]

中断于第 630 行，错误: No objects to concatenate
完成处理 630 行.
